In [6]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# ✅ Rutas dinámicas correctas para cualquier sistema operativo
base_path = os.path.join(os.getcwd(), "dataset")
train_dir = os.path.join(base_path, "Training")
test_dir = os.path.join(base_path, "Testing")

print("Ruta entrenamiento:", train_dir)
print("Ruta prueba:", test_dir)
print("¿Existen?:", os.path.exists(train_dir), os.path.exists(test_dir))


Ruta entrenamiento: c:\Users\Lenovo\Desktop\ClassifierModelReImagesTumoresCerebrales\dataset\Training
Ruta prueba: c:\Users\Lenovo\Desktop\ClassifierModelReImagesTumoresCerebrales\dataset\Testing
¿Existen?: True True


In [2]:
# Generadores de datos
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [3]:
# Modelo CNN
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])


c:\Users\Lenovo\Desktop\ClassifierModelReImagesTumoresCerebrales\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
# Compilar y entrenar
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_gen, validation_data=test_gen, epochs=10)

# Guardar modelo
model.save("brain_tumor_cnn.h5")


c:\Users\Lenovo\Desktop\ClassifierModelReImagesTumoresCerebrales\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 275s 2s/step - accuracy: 0.5610 - loss: 0.9948 - val_accuracy: 0.7818 - val_loss: 0.5067
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 207s 862ms/step - accuracy: 0.8286 - loss: 0.4686 - val_accuracy: 0.8322 - val_loss: 0.3877
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 171s 687ms/step - accuracy: 0.8730 - loss: 0.3491 - val_accuracy: 0.8780 - val_loss: 0.2959
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 115s 641ms/step - accuracy: 0.8933 - loss: 0.2726 - val_accuracy: 0.8642 - val_loss: 0.3087
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 124s 695ms/step - accuracy: 0.9296 - loss: 0.2019 - val_accuracy: 0.9161 - val_loss: 0.2090
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 109s 611ms/step - accuracy: 0.9430 - loss: 0.1540 - val_accuracy: 0.9314 - val_loss: 0.1739
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 128s 710ms/step - accuracy: 0.9506 - loss: 0.1303 - val_accuracy: 0.9527 - val_loss: 0.1534
Epoch 8/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 112s 624ms/step - accuracy: 0.9602 - lo

In [5]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.title("Precisión"), plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title("Pérdida"), plt.legend()
plt.tight_layout()
plt.savefig("training_result.png")
plt.close()

# Evaluación
test_gen.reset()
predictions = model.predict(test_gen)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

report = classification_report(true_classes, predicted_classes, target_names=class_labels)
matrix = confusion_matrix(true_classes, predicted_classes)

print(report)
print(matrix)


41/41 ━━━━━━━━━━━━━━━━━━━━ 7s 163ms/step
              precision    recall  f1-score   support

      glioma       0.96      0.95      0.95       300
  meningioma       0.94      0.90      0.92       306
     notumor       0.97      1.00      0.99       405
   pituitary       0.97      1.00      0.98       300

    accuracy                           0.96      1311
   macro avg       0.96      0.96      0.96      1311
weighted avg       0.96      0.96      0.96      1311

[[284  15   0   1]
 [ 12 274  12   8]
 [  0   0 405   0]
 [  0   1   0 299]]
